In [1]:
import pickle
import pandas as pd
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
def load_data(database_filepath):
    """Loads the data from a database file and returns the target."""

    engine = create_engine(f'sqlite:///{database_filepath}')
    df = pd.read_sql_table('emissions', engine)

    return df

In [3]:
df = load_data('../data/emissions.db')
df.shape

(45511, 29)

In [8]:
def get_categorical_lists(df):
    """
    INPUT - df - fuel emissions dataframe
    
    OUTPUT
    mf - list of possible manufacturers
    tmt - list of possible transmission types
    tm - list of possible transmissions
    ft - list of possible fuel types
    """
    
    mfs = df['manufacturer'].unique()
    tmts = df['transmission_type'].unique()
    tms = df['transmission'].unique()
    fts = df['fuel_type'].unique()
    
    return mfs, tmts, tms, fts

In [9]:
mfs, tmts, tms, fts = get_categorical_lists(df)
mfs, tmts, tms, fts

(array(['Alfa Romeo', 'Aston Martin Lagonda', 'Audi', 'BMW', 'Cadillac',
        'Chevrolet', 'Chrysler Jeep', 'Citroen', 'Daewoo Cars', 'Daihatsu',
        'Ferrari', 'Fiat', 'Ford', 'Honda', 'Hyundai', 'Isuzu',
        'Jaguar Cars', 'Kia', 'Lamborghini', 'Land Rover', 'Lexus',
        'Lotus', 'LTI', 'Maserati', 'Mazda', 'Mercedes-Benz', 'Metrocab',
        'Micro Compact Car', 'Mitsubishi', 'Nissan', 'Perodua', 'Peugeot',
        'Porsche', 'Proton', 'Renault', 'Rolls-Royce',
        'Rover Group Limited', 'Saab', 'Seat', 'Skoda', 'Subaru', 'Suzuki',
        'Tata', 'Toyota', 'Vauxhall', 'Volkswagen', 'Volvo',
        'MG Rover Group', 'Mini', 'Morgan Motor Company', 'Bentley Motors',
        'Smart', 'SsangYong', 'Corvette', 'Dodge', 'Hummer', 'Abarth',
        'Infiniti', 'MG Motors Uk', 'McLaren', 'Dacia', 'MG Motors UK'],
       dtype=object),
 array(['Manual', 'Automatic', None], dtype=object),
 array(['M5', 'SAT5', 'M6', 'A4', 'A5', 'QM6', 'QA5', 'QM5', 'QM', 'QA',
        'F

In [14]:
def split_data(df):
    """
    INPUT - df - full car emissions df
    
    OUTPUT
    X_train, X_test, y_train, y_test - test and train sets
    X_pop - parameter matrix for given tax band labels
    X_nan - parameter matrix of missing tax band values which cannot be used for training and testing the model. y_nan would be an empty
    """
    
    df_pop = df.dropna(subset=['tax_band'])
    X = df_pop.drop(columns='tax_band')
    y = df_pop['tax_band']
    
    X_train, X_test, y_train, y_test, = train_test_split(X, y)
    
    df_nan = df[df['tax_band'].isnull()]
    X_nan = df_nan.drop(columns='tax_band')

    return X_train, X_test, y_train, y_test, df_pop, X_nan

In [15]:
X_train, X_test, y_train, y_test, df_pop, X_nan = split_data(df)
X_train.shape, X_test.shape, y_train.shape, y_test.shape, df_pop.shape, X_nan.shape

((5877, 28), (1960, 28), (5877,), (1960,), (7837, 29), (37674, 28))

In [11]:
df_pop['manufacturer'].value_counts()

BMW                     890
Volkswagen              832
Mercedes-Benz           770
Vauxhall                533
Audi                    473
Ford                    441
Volvo                   432
Peugeot                 321
Kia                     313
Skoda                   306
Honda                   227
Citroen                 219
Mini                    158
Porsche                 146
Toyota                  145
Seat                    143
Renault                 139
Chevrolet               138
Hyundai                 123
Chrysler Jeep           109
Nissan                  107
Subaru                  104
Fiat                     89
Mazda                    88
Mitsubishi               85
Suzuki                   65
Land Rover               45
Aston Martin Lagonda     40
Smart                    40
Lexus                    37
Alfa Romeo               33
Jaguar Cars              32
SsangYong                30
Infiniti                 23
Bentley Motors           21
Ferrari             

In [12]:
small_mf = df_pop['manufacturer'].value_counts() <= 30
small_mf[small_mf].index

Index(['SsangYong', 'Infiniti', 'Bentley Motors', 'Ferrari', 'Maserati',
       'Corvette', 'Lotus', 'Morgan Motor Company', 'Rolls-Royce', 'Abarth',
       'Dacia', 'McLaren', 'Perodua', 'MG Motors UK', 'MG Motors Uk', 'LTI'],
      dtype='object')

In [13]:
X = X_train.copy()
group_small = lambda row : 'Other' if row in small_mf[small_mf].index else row
X['manufacturer'] = X['manufacturer'].apply(group_small)
X['manufacturer'].unique()

array(['Other', 'Ford', 'Mitsubishi', 'Peugeot', 'Audi', 'BMW',
       'Mercedes-Benz', 'Volkswagen', 'Seat', 'Chevrolet', 'Smart',
       'Honda', 'Vauxhall', 'Alfa Romeo', 'Hyundai', 'Kia', 'Nissan',
       'Volvo', 'Mazda', 'Chrysler Jeep', 'Renault', 'Skoda', 'Lexus',
       'Suzuki', 'Citroen', 'Fiat', 'Subaru', 'Porsche', 'Toyota', 'Mini',
       'Land Rover', 'Aston Martin Lagonda', 'Jaguar Cars'], dtype=object)

In [14]:
small_tm = df_pop['transmission'].value_counts() <= 30
small_tm[small_tm].index

Index(['M7', 'QA6', '5AT', 'SAT5', '4AT', 'AMT5', 'A6-AWD', 'A6x2', 'ET5',
       'ASM', 'DCT7', 'M6-AWD', 'SAT6', 'M6x2', '7SP. SSG', 'MultiDriv',
       'MultiDrive', 'A8-AWD', '5MTx2', 'Multi5', 'M5x2', 'A5-AWD', 'MTA5',
       'Multi6', 'S/A6', 'M8'],
      dtype='object')

In [15]:
df_pop['transmission_type'].value_counts()

Manual       4275
Automatic    3221
Name: transmission_type, dtype: int64

In [16]:
df_pop['fuel_type'].value_counts()

Diesel                      3960
Petrol                      3747
Petrol Hybrid                 62
Diesel Electric               22
Petrol / E85 (Flex Fuel)      16
Petrol Electric               13
Electricity                    7
Electricity/Petrol             5
CNG                            4
Electricity/Diesel             1
Name: fuel_type, dtype: int64

In [17]:
small_ft = df_pop['fuel_type'].value_counts() <= 30
small_ft[small_ft].index

Index(['Diesel Electric', 'Petrol / E85 (Flex Fuel)', 'Petrol Electric',
       'Electricity', 'Electricity/Petrol', 'CNG', 'Electricity/Diesel'],
      dtype='object')

In [94]:
class EmissionsTransformer(BaseEstimator, TransformerMixin):
    """
    Adds dummies to categorical columns and removes the original ones
    """

    def __init__(self, mfs, tmts, tms, fts):
        self.mfs, self.tmts, self.tms, self.fts = mfs, tmts, tms, fts

    def drop_columns(self, X):
        """Dropping irrelevant columns from the data set"""

        irrelevant_numeric = [
            'urban_metric', 'extra_urban_metric', 'urban_imperial',
            'extra_urban_imperial', 'combined_imperial', 'thc_nox_emissions',
            'fuel_cost_6000_miles', 'standard_12_months', 'standard_6_months',
            'first_year_12_months', 'first_year_6_months',
        ]
        X = X.drop(irrelevant_numeric, axis=1)

        irrelevant_categorical = ['model', 'description']
        X = X.drop(irrelevant_categorical, axis=1)

        return X

    def fill_columns(self, X):
        """Filling the numeric columns with the mean of these columns"""

        relevant_numeric = [
            'year', 'euro_standard', 'noise_level', 'engine_capacity',
            'combined_metric', 'fuel_cost_12000_miles', 'co2', 'thc_emissions',
            'co_emissions', 'nox_emissions', 'particulates_emissions',
        ]
        X[relevant_numeric] = X[relevant_numeric].fillna(X.mean())

        return X

    def add_dummies(self, X):
        """Add dummies for every possible value in the data set"""

        for mf in self.mfs:
            X[f'manufacturer_{mf}'] = (X['manufacturer'] == mf).astype(int)
        for tmt in self.tmts:
            X[f'manufacturer_{tmt}'] = (X['manufacturer'] == tmt).astype(int)
        for tm in self.tms:
            X[f'manufacturer_{tm}'] = (X['manufacturer'] == tm).astype(int)
        for ft in self.fts:
            X[f'manufacturer_{ft}'] = (X['manufacturer'] == ft).astype(int)

        X = X.drop(['manufacturer', 'transmission', 'transmission_type', 'fuel_type'], axis=1)
        return X

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = self.drop_columns(X)
        X = self.fill_columns(X)
        X = self.add_dummies(X)
        return X


In [81]:
transformer = EmissionsTransformer(mfs, tmts, tms, fts)
transformer.transform(X_train)

,year,euro_standard,engine_capacity,combined_metric,noise_level,co2,thc_emissions,co_emissions,nox_emissions,particulates_emissions,...,manufacturer_Petrol Hybrid,manufacturer_CNG,manufacturer_Petrol Electric,manufacturer_LPG / Petrol,manufacturer_Petrol / E85 (Flex Fuel),manufacturer_Petrol / E85,manufacturer_Diesel Electric,manufacturer_Electricity/Petrol,manufacturer_Electricity,manufacturer_Electricity/Diesel
41076,2012,5,1999.0,8.3,73.0,193,65.000000,524.0,22.0,0.590883,...,0,0,0,0,0,0,0,0,0,0
39195,2012,6,2191.0,5.2,71.1,136,42.492718,97.0,68.0,0.200000,...,0,0,0,0,0,0,0,0,0,0
38621,2012,5,1596.0,6.0,70.0,139,56.000000,340.0,34.0,0.590883,...,0,0,0,0,0,0,0,0,0,0
38237,2012,5,4395.0,11.4,72.0,266,27.000000,380.0,18.0,0.590883,...,0,0,0,0,0,0,0,0,0,0
41797,2013,5,4395.0,10.4,72.0,243,49.000000,560.0,22.0,0.590883,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44391,2013,5,1956.0,4.7,71.0,124,42.492718,218.0,169.0,1.400000,...,0,0,0,0,0,0,0,0,0,0
44420,2013,5,1686.0,6.4,70.0,168,42.492718,41.0,163.0,0.500000,...,0,0,0,0,0,0,0,0,0,0
40535,2012,5,1398.0,6.1,72.0,143,50.000000,813.0,16.0,0.590883,...,0,0,0,0,0,0,0,0,0,0
41450,2013,5,4163.0,14.2,74.0,332,46.000000,272.0,40.0,0.590883,...,0,0,0,0,0,0,0,0,0,0


In [82]:
def build_model(mfs, tmts, tms, fts):
    """Describes the model used on the data, consisting of NLP transformers and
    an individual classifier of each category."""

    pipeline = Pipeline([
        ('et', EmissionsTransformer(mfs, tmts, tms, fts)),
        ('clf', RandomForestClassifier()),
    ])

    parameters = {
        'clf__criterion': ['gini', 'entropy'],
    }

    model = GridSearchCV(pipeline, param_grid=parameters)

    return model

In [85]:
model = build_model(mfs, tmts, tms, fts)

In [86]:
model.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('et',
   EmissionsTransformer(fts=array(['Petrol', 'Diesel', 'LPG', 'Petrol Hybrid', 'CNG',
          'Petrol Electric', 'LPG / Petrol', 'Petrol / E85 (Flex Fuel)',
          'Petrol / E85', 'Diesel Electric', 'Electricity/Petrol',
          'Electricity', 'Electricity/Diesel'], dtype=object),
                        mfs=array(['Alfa Romeo', 'Aston Martin Lagonda', 'Audi', 'BMW', 'Cadillac',
          'Chevrolet', 'Chrysler Jeep', 'Citroen', 'Daewoo Cars', '...
          'Electric', 'm5', 'SMG 7', '6MT', '5A/Tx2', 'M6x2', '5AT', '6AMT',
          '5 AMT', 'MTA5', 'i-SHIFT', 'A8', 'DCT7', 'M6-AWD', 'S6', 'A6x2',
          'D7', 'M7', 'A5-AWD', 'MultiDrive', 'MTA6', 'QD7', 'A6-AWD', 'DM6',
          'MCVT', 'Semi-Auto', 'AM5', 'Multi6', 'MultiDriv', 'DCT6', 'ET5',
          'QA8', '6AT', 'AMT6', 'AMT5', '7SP. SSG', None, 'M8', 'A8-AWD'],
         dtype=object),
                        tmts=array(['Manual

In [87]:
def evaluate_model(model, X_test, y_test):
    """Shows the accuracy, precision, and recall of the model."""

    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print(model.best_params_)

In [88]:
def save_model(model, model_filepath):
    """Saves the model as a pickle file"""

    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)

In [95]:
database_filepath = '../data/emissions.db'
model_filepath = 'classifier.pkl'

print('Loading data...')
df = load_data(database_filepath)

mfs, tmts, tms, fts = get_categorical_lists(df)

print('Splitting data...')
X_train, X_test, y_train, y_test, df_pop, X_nan = split_data(df)

print('Building model...')
model = build_model(mfs, tmts, tms, fts)

print('Training model...')
model.fit(X_train, y_train)

print('Evaluating model...')
evaluate_model(model, X_test, y_test)

print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath)

print('Trained model saved!')

Loading data...
Splitting data...
Building model...
Training model...
Evaluating model...
              precision    recall  f1-score   support

           A       1.00      0.99      0.99        89
           B       0.96      0.98      0.97        81
           C       0.99      0.99      0.99       206
           D       1.00      0.99      1.00       216
           E       0.99      0.99      0.99       273
           F       0.99      1.00      0.99       233
           G       0.99      1.00      0.99       266
           H       0.98      0.99      0.99       127
           I       0.98      0.94      0.96        90
           J       0.97      0.98      0.98       118
           K       1.00      0.99      1.00       114
           L       0.97      1.00      0.98        64
           M       1.00      0.99      0.99        83

    accuracy                           0.99      1960
   macro avg       0.99      0.99      0.99      1960
weighted avg       0.99      0.99      0.99 